In [1]:
!pip install -q --upgrade pip
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html


In [2]:
!pip uninstall jaxlib
!pip install -U jaxlib==0.4.2+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Found existing installation: jaxlib 0.4.2+cuda11.cudnn82
Uninstalling jaxlib-0.4.2+cuda11.cudnn82:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/jaxlib-0.4.2+cuda11.cudnn82.dist-info/*
    /usr/local/lib/python3.8/dist-packages/jaxlib/*
Proceed (Y/n)? Y
  Successfully uninstalled jaxlib-0.4.2+cuda11.cudnn82
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.4.2%2Bcuda11.cudnn82-cp38-cp38-manylinux2014_x86_64.whl (164.4 MB)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

#/content/drive/MyDrive/MasterarbeitData

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [7]:
from utils import *

In [8]:
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [9]:
# preproces data
dataset = TUDataset(root="Masterarbeit", name="MUTAG")

# make lists of dense adjacency matrix, node feature array and y
graps_node_features = list()
As = list()
ys = list()

for data in dataset:
  As.append(to_dense(data.edge_index, len(data.x)))
  graps_node_features.append(np.array(data.x))
  ys.append(np.array(data.y))


# get the initial edge feature matrix
# when the graph specification has only node features
# as a batch x nodes x nodes x channels array

nb_graphs = len(graps_node_features)
max_nodes = len(max(graps_node_features, key=lambda x: len(x)))
feature_dim = graps_node_features[0][0].shape[0]

graphs_edge_features = np.zeros((nb_graphs, max_nodes, max_nodes, feature_dim))

for k, node_features in enumerate(graps_node_features):
    for i, node_feature in enumerate(node_features):
        graphs_edge_features = graphs_edge_features.at[k,i,i,:].set(node_feature)

# calculate the pattern array for a edge feature matrix
# of shape batch x nodes x nodes x channels array

As = [zero_append(a, (max_nodes, max_nodes)) for a in As]
As = np.array(As)

# use r = 1
As_3 = r_power_adjacency_matrix(As, 3)
As_3_int = neigbourhood_intersections(As_3)
As_3_pattern = np.transpose(np.array(np.nonzero(As_3_int)))

As_1 = As
As_1_int = neigbourhood_intersections(As_1)
As_1_pattern = np.transpose(np.array(np.nonzero(As_1_int)))

print(As_3_pattern.shape)
print(graphs_edge_features.shape)

(353051, 4)
(188, 28, 28, 7)


In [10]:
import os
newpath = "/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [11]:
# save preprocessed data

np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/ys", ys)
np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/As_3_pattern", As_3_pattern)
np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/graphs_edge_features", graphs_edge_features)

np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/As_1_pattern", As_1_pattern)

In [22]:
# load preprocessed data

ys = np.load(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/ys.npy")
As_3_pattern = np.load(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/As_3_pattern.npy")
As_1_pattern = np.load(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/As_1_pattern.npy")
graphs_edge_features = np.load(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/graphs_edge_features.npy")

print(As_1_pattern.shape)
print(As_3_pattern.shape)
print(graphs_edge_features.shape)

(18298, 4)
(353051, 4)
(188, 28, 28, 7)


In [14]:
import shutil
shutil.rmtree("/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG")

In [28]:
# split the preprocessed data into pieces, to save memory

import os

size = 20
r = 9

newpath = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/size_{size}_range_{r}" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

for i in range(r):
  for j in range(r):
    x1_batch_range = list(range(i*size,(1+i)*size))
    x2_batch_range = list(range(j*size,(1+j)*size))

    x1 = graphs_edge_features[x1_batch_range,:]
    x2 = graphs_edge_features[x2_batch_range,:]

    p1 = As_1_pattern[column_in_values(As_1_pattern[:,0], np.array(x1_batch_range)),:]
    p2 = As_1_pattern[column_in_values(As_1_pattern[:,0], np.array(x2_batch_range)),:]  

    np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/size_{size}_range_{r}/x1_{i*size}_{(1+i)*size}", x1)
    np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/size_{size}_range_{r}/x2_{j*size}_{(1+j)*size}", x2)
    np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/size_{size}_range_{r}/p1_{i*size}_{(1+i)*size}", p1)
    np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/size_{size}_range_{r}/p2_{j*size}_{(1+j)*size}", p2)